In [1]:
import cv2
import os
import json

# Videoların bulunduğu klasör
video_folder = "İşaretDiliVideo"
# Çıktı framelerinin kaydedileceği klasör
output_frames_folder = "frames"
# JSON dosyası adı
output_json = "all_frames.json"

# Çıkış klasörünü oluştur
os.makedirs(output_frames_folder, exist_ok=True)

# JSON yapısını tutacak liste
all_frames = []

# Klasördeki tüm videoları işle
for video_file in os.listdir(video_folder):
    if video_file.endswith(".mp4"):  # Sadece mp4 dosyalarını al
        video_path = os.path.join(video_folder, video_file)
        video_name = os.path.splitext(video_file)[0]  # Video ismini al

        cap = cv2.VideoCapture(video_path)
        fps = int(cap.get(cv2.CAP_PROP_FPS))  # FPS değerini al
        frame_id = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Frame dosya adını oluştur
            frame_filename = f"{video_name}_frame_{frame_id:04d}.jpg"
            frame_path = os.path.join(output_frames_folder, frame_filename)

            # Frame'i kaydet
            cv2.imwrite(frame_path, frame)

            # JSON'a eklenmek üzere frame bilgisi
            all_frames.append({
                "frame_id": frame_id,
                "video_name": video_name,
                "image_path": frame_path.replace("\\", "/"),  # Windows uyumu için
                "timestamp": frame_id / fps,  # Zaman damgası (saniye cinsinden)
                "keypoints": []  # İstersen buraya pose keypoints ekleyebilirsin
            })

            frame_id += 1

        cap.release()

# Tüm frameleri JSON dosyasına yaz
with open(output_json, "w") as json_file:
    json.dump({"frames": all_frames}, json_file, indent=4)

print(f"Tüm videolar framelendi ve JSON olarak kaydedildi: {output_json}")

Tüm videolar framelendi ve JSON olarak kaydedildi: all_frames.json


In [ ]:
import os
import json
import cv2
import speech_recognition as sr
import re

# JSON dosyasını oku ve cümleyi anlamaya çalış
def get_video_frames(json_file, sentence):
    with open(json_file, "r") as f:
        data = json.load(f)
    
    words = sentence.lower().split()  # Cümleyi küçük harfe dönüştürüp kelimelere ayırıyoruz
    frame_list = []
    added_videos = set()  # Zaten eklenmiş videoları takip etmek için bir set
    
    # Video adlarında özel karakterler ve boşlukları temizleme fonksiyonu
    def clean_string(string):
        return re.sub(r'[^a-zA-Z0-9ğüşöçİıâîîıéèíâ]', '', string.lower().strip())

    # Her kelimeyi kontrol et
    for word in words:
        word_found = False  # Kelimeyi bulduğumuzu takip etmek için bir değişken
        for item in data["frames"]:
            video_name = clean_string(item["video_name"])  # JSON dosyasındaki video_name'i temizle
            clean_word = clean_string(word)  # Kullanıcının söylediği kelimeyi temizle
            if clean_word == video_name and video_name not in added_videos:
                img_path = item["image_path"].replace("\\", "/")  # Yolu düzelt
                if os.path.exists(img_path):
                    img = cv2.imread(img_path)
                    frame_list.append(img)
                    added_videos.add(video_name)  # Bu video ismini ekledik
                    print(f"Kelime '{word}' için video bulundu: {video_name}")
                    word_found = True
                    break  # Aynı kelimeyi tekrar arama, sadece ilk bulduğunda ekle
        if not word_found:
            print(f"Kelime '{word}' için frame bulunamadı. Bu kelimeye ait video adı ve frame yolu kontrol edilebilir.")
    
    if not frame_list:
        print("No frames loaded. Check file paths and JSON structure.")
    
    return frame_list

def create_video(frames, output_video, fps=30):
    if not frames:
        print("No frames to create a video.")
        return
    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))
    for frame in frames:
        out.write(frame)
    out.release()
    print(f"Video saved at {output_video}")

def recognize_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Konuşmanızı bekliyorum...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
    
    try:
        text = recognizer.recognize_google(audio, language="tr-TR")
        print("Algılanan metin:", text)
        return text.lower()
    except sr.UnknownValueError:
        print("Ses anlaşılamadı.")
    except sr.RequestError:
        print("Ses tanıma servisine ulaşılamadı.")
    return ""

def main():
    json_file = "frames\all_frames.json"  # JSON dosyanızın yolu
    sentence = recognize_speech()
    if sentence:
        video_frames = get_video_frames(json_file, sentence)
        create_video(video_frames, "output_video.mp4")

if __name__ == "__main__":
    main()

Konuşmanızı bekliyorum...
Algılanan metin: Merhaba ben
Kelime 'merhaba' için video bulundu: merhaba
Kelime 'ben' için video bulundu: ben
Video saved at output_video.mp4


In [29]:
import json
import cv2
import imageio
import os
import numpy as np
import speech_recognition as sr

def mikrofondan_metne_cevir():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Konuşmaya başlayın...")
        recognizer.adjust_for_ambient_noise(source)
        ses_verisi = recognizer.listen(source)
    try:
        metin = recognizer.recognize_google(ses_verisi, language="tr-TR")
        print("Algılanan metin:", metin)
        kelimeler = metin.lower().split()  # Küçük harfe dönüştür ve kelimelere ayır
        return [kelime.strip(",.!?") for kelime in kelimeler]  # Noktalama işaretlerini temizle
    except sr.UnknownValueError:
        print("Ses anlaşılamadı.")
    except sr.RequestError:
        print("Servise ulaşılamadı.")
    return []

def json_dosyasindan_animasyon_olustur(json_dosyasi, kelimeler, cikis_gif, frame_klasoru):
    image_list = []

    # JSON dosyasını açalım ve yapısını kontrol edelim
    if not os.path.exists(json_dosyasi):
        print("JSON dosyası bulunamadı!")
        return
    
    with open(json_dosyasi, "r", encoding="utf-8") as f:
        try:
            data = json.load(f)  # JSON'u yükle
            print("JSON içeriği başarıyla yüklendi.")
        except json.JSONDecodeError:
            print("JSON dosyasının formatı hatalı!")
            return

    # JSON dosyasının "frames" anahtarına sahip olduğundan emin olalım
    if "frames" in data:
        frames = data["frames"]  # 'frames' listesine eriş
        print(f"Toplam {len(frames)} frame bulundu.")
        
        # JSON içindeki 'video_name' ile kelimeyi karşılaştırıyoruz
        for kelime in kelimeler:
            kelime_bulundu = False
            for item in frames:
                video_name = item.get("video_name", "").lower()  # video_name küçük harfe dönüştürülür
                if video_name == kelime.lower():  # Kelime ile karşılaştırma yapılır
                    kelime_bulundu = True
                    image_path = item.get("image_path", "")
                    if image_path:
                        # Görselin tam yolu
                        # Burada 'frames' klasörünü zaten image_path içinde var, bir daha eklemiyoruz
                        full_image_path = os.path.join(frame_klasoru, image_path)
                        print(f"Görselin tam yolu: {full_image_path}")  # Yolu yazdırıyoruz
                        if os.path.exists(full_image_path):
                            print(f"Görsel bulundu: {full_image_path}")  # Görselin bulunduğu yol
                            img = cv2.imread(full_image_path)
                            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                            image_list.append(np.array(img))
                        else:
                            print(f"Uyarı: Görsel bulunamadı: {full_image_path}")
                    break  # İlk bulunan kelimeyi ekledikten sonra diğerlerine bakmaya gerek yok

            if not kelime_bulundu:
                print(f"Uyarı: '{kelime}' için JSON dosyasında bir kayıt bulunamadı.")
    else:
        print("JSON dosyasında 'frames' anahtarı bulunamadı.")
        return

    if image_list:
        min_shape = min([img.shape for img in image_list], key=lambda x: x[0] * x[1])
        image_list = [cv2.resize(img, (min_shape[1], min_shape[0])) for img in image_list]
        imageio.mimsave(cikis_gif, image_list, fps=10)
        print(f"Animasyon kaydedildi: {cikis_gif}")
    else:
        print("Hiçbir geçerli görüntü bulunamadı.")

# Kullanım
json_dosyasi = "all_frames.json"  # JSON dosyasının tam yolu
frame_klasoru = "frames"  # Framelerin bulunduğu klasörün yolu
cikis_gif = "animation.gif"  # Çıkış animasyonu

kelimeler = mikrofondan_metne_cevir()
json_dosyasindan_animasyon_olustur(json_dosyasi, kelimeler, cikis_gif, frame_klasoru)


Konuşmaya başlayın...
Algılanan metin: Merhaba ben
JSON içeriği başarıyla yüklendi.
Toplam 661 frame bulundu.
Görselin tam yolu: frames\frames/Merhaba_frame_0000.jpg
Uyarı: Görsel bulunamadı: frames\frames/Merhaba_frame_0000.jpg
Görselin tam yolu: frames\frames/Ben_frame_0000.jpg
Uyarı: Görsel bulunamadı: frames\frames/Ben_frame_0000.jpg
Hiçbir geçerli görüntü bulunamadı.


In [7]:
#sevval
import os
import json
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Veri yolları
json_path = "all_frames.json"

# Veri setini yükleme
def load_dataset(json_path, image_size=(64, 64)):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    images = []
    labels = []
    label_map = {word: idx for idx, word in enumerate(data.keys())}  # Kelime - ID eşlemesi

    for word, frames in data.items():
        for frame in frames:
            img_path = frame["image_path"].replace("\\", "/")  # Yol formatını düzeltme
            if os.path.exists(img_path):
                img = cv2.imread(img_path)
                img = cv2.resize(img, image_size)  # Görselleri aynı boyuta getir
                img = img / 255.0  # Normalizasyon
                images.append(img)
                labels.append(label_map[word])
    
    images = np.array(images)
    labels = to_categorical(np.array(labels), num_classes=len(data))  # One-hot encoding

    return images, labels, label_map

# Veri yükleme
X, y, label_map = load_dataset(json_path)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Toplam veri: {len(X)}, Eğitim: {len(X_train)}, Test: {len(X_test)}")






from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# CNN Modeli
def build_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')  # Çok sınıflı çıktı
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Model oluşturma
input_shape = X_train.shape[1:]
num_classes = len(label_map)
model = build_model(input_shape, num_classes)

# Model eğitme
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

# Modeli kaydetme
model.save("sign_language_model.h5")




# Modeli yükleme
model = tf.keras.models.load_model("sign_language_model.h5")

def predict_word(image_path, model, label_map, image_size=(64, 64)):
    img = cv2.imread(image_path)
    img = cv2.resize(img, image_size) / 255.0
    img = np.expand_dims(img, axis=0)

    prediction = model.predict(img)
    predicted_label = np.argmax(prediction)

    for word, idx in label_map.items():
        if idx == predicted_label:
            return word
    return None

def generate_video(sentence, model, label_map, output_video="output.mp4", fps=30):
    words = sentence.lower().split()
    frame_list = []

    for word in words:
        for frame in data.get(word, []):
            img_path = frame["image_path"].replace("\\", "/")
            if os.path.exists(img_path):
                predicted_word = predict_word(img_path, model, label_map)
                if predicted_word == word:
                    img = cv2.imread(img_path)
                    frame_list.append(img)

    if frame_list:
        height, width, _ = frame_list[0].shape
        out = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))

        for frame in frame_list:
            out.write(frame)

        out.release()
        print(f"Video kaydedildi: {output_video}")
    else:
        print("Hiçbir frame bulunamadı!")

# Kullanıcıdan cümle al
sentence = "merhaba ben üniversite öğrencisiyim"
generate_video(sentence, model, label_map)

AttributeError: partially initialized module 'jax' has no attribute 'version' (most likely due to a circular import)

In [33]:
#eslem
import cv2
import csv
import os

def extract_frames_from_video(video_path, video_name, output_folder="frames"):
    """
    Verilen video yolundan frame'leri çıkarır ve her bir frame için bir dosya kaydeder.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Videoyu aç
    cap = cv2.VideoCapture(video_path)
    frame_id = 0
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Frame'i kaydet
        frame_filename = f"{video_name}_frame_{frame_id:04d}.jpg"
        frame_filepath = os.path.join(output_folder, frame_filename)
        cv2.imwrite(frame_filepath, frame)
        
        # Frame bilgilerini kaydet
        frames.append({
            'video_name': video_name,
            'frame_id': frame_id,
            'image_path': frame_filepath,
            'timestamp': frame_id / cap.get(cv2.CAP_PROP_FPS)
        })
        
        frame_id += 1

    cap.release()
    return frames

def save_frames_to_csv(videos, output_csv="video_frames.csv"):
    """
    Video listesine ait frame bilgilerini bir CSV dosyasına kaydeder.
    """
    with open(output_csv, mode='w', newline='') as file:
        fieldnames = ['video_name', 'frame_id', 'image_path', 'timestamp']
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        
        writer.writeheader()
        
        for video in videos:
            for frame in video['frames']:
                writer.writerow(frame)

def main():
    video_files = [
        "İşaretDiliVideo\Merhaba.mp4",  # Video 1'in yolu
        "İşaretDiliVideo\Ben.mp4",  # Video 2'nin yolu
        "İşaretDiliVideo\Üniversite.mp4" ,  # Video 3'ün yolu
        "İşaretDiliVideo\Öğrencisiyim.mp4",
        "İşaretDiliVideo\Şimdi.mp4",
        "İşaretDiliVideo\Proje.mp4",
        "İşaretDiliVideo\geliştiriyorum.mp4"

    ]
    
    videos = []
    
    for video_file in video_files:
        video_name = os.path.splitext(os.path.basename(video_file))[0]  # Video ismini al
        frames = extract_frames_from_video(video_file, video_name)  # Frame'leri çıkar
        videos.append({
            'video_name': video_name,
            'frames': frames
        })
    
    save_frames_to_csv(videos)  # Frame bilgilerini CSV'ye kaydet

if __name__ == "__main__":
    main()

<>:57: SyntaxWarning: invalid escape sequence '\M'
<>:58: SyntaxWarning: invalid escape sequence '\B'
<>:62: SyntaxWarning: invalid escape sequence '\P'
<>:63: SyntaxWarning: invalid escape sequence '\g'
<>:57: SyntaxWarning: invalid escape sequence '\M'
<>:58: SyntaxWarning: invalid escape sequence '\B'
<>:62: SyntaxWarning: invalid escape sequence '\P'
<>:63: SyntaxWarning: invalid escape sequence '\g'
C:\Users\hp\AppData\Local\Temp\ipykernel_9240\2434042197.py:57: SyntaxWarning: invalid escape sequence '\M'
  "İşaretDiliVideo\Merhaba.mp4",  # Video 1'in yolu
C:\Users\hp\AppData\Local\Temp\ipykernel_9240\2434042197.py:58: SyntaxWarning: invalid escape sequence '\B'
  "İşaretDiliVideo\Ben.mp4",  # Video 2'nin yolu
C:\Users\hp\AppData\Local\Temp\ipykernel_9240\2434042197.py:62: SyntaxWarning: invalid escape sequence '\P'
  "İşaretDiliVideo\Proje.mp4",
C:\Users\hp\AppData\Local\Temp\ipykernel_9240\2434042197.py:63: SyntaxWarning: invalid escape sequence '\g'
  "İşaretDiliVideo\geliştiri

In [34]:
import csv

def load_csv_data(csv_file):
    frames = []
    with open(csv_file, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            frames.append({
                'video_name': row['video_name'],
                'frame_id': int(row['frame_id']),
                'image_path': row['image_path'],
                'timestamp': float(row['timestamp'])
            })
    return frames

# CSV verisini yükle
frames_data = load_csv_data('video_frames.csv')
print(frames_data[:5])  # İlk 5 frame'i kontrol et

[{'video_name': 'Merhaba', 'frame_id': 0, 'image_path': 'frames\\Merhaba_frame_0000.jpg', 'timestamp': 0.0}, {'video_name': 'Merhaba', 'frame_id': 1, 'image_path': 'frames\\Merhaba_frame_0001.jpg', 'timestamp': 0.033569739952718676}, {'video_name': 'Merhaba', 'frame_id': 2, 'image_path': 'frames\\Merhaba_frame_0002.jpg', 'timestamp': 0.06713947990543735}, {'video_name': 'Merhaba', 'frame_id': 3, 'image_path': 'frames\\Merhaba_frame_0003.jpg', 'timestamp': 0.10070921985815603}, {'video_name': 'Merhaba', 'frame_id': 4, 'image_path': 'frames\\Merhaba_frame_0004.jpg', 'timestamp': 0.1342789598108747}]


In [37]:
import csv
import cv2
import os

# CSV dosyasını okuyup frame'leri bulma
def get_video_frames_from_csv(csv_file, sentence):
    frames = []
    words = sentence.lower().split()  # Cümleyi küçük harfe dönüştürüp kelimelere ayırıyoruz
    
    with open(csv_file, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            video_name = row['video_name'].lower()  # CSV'deki video_name'i küçük harfe dönüştür
            if video_name in words:  # Cümledeki kelimelerle eşleşmeye çalışıyoruz
                img_path = row['image_path'].replace("\\", "/")  # Yol düzeltmesi
                if os.path.exists(img_path):  # Dosya var mı kontrol et
                    img = cv2.imread(img_path)
                    frames.append(img)
    
    return frames

# Videoyu oluşturma fonksiyonu
def create_video(frames, output_video, fps=30):
    if not frames:
        print("No frames to create a video.")
        return
    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))
    for frame in frames:
        out.write(frame)
    out.release()
    print(f"Video saved at {output_video}")

# Ana fonksiyon
def main():
    csv_file = "video_frames.csv"  # CSV dosyanızın yolu
    sentence = "Merhaba Ben"  # Kullanıcıdan alınan cümle
    
    # Cümleyi söylesek ve onu kullanarak frame'leri alalım
    video_frames = get_video_frames_from_csv(csv_file, sentence)
    
    # Tüm frame'lerle video oluşturalım
    create_video(video_frames, "output_video.mp4")

if __name__ == "__main__":
    main()

Video saved at output_video.mp4
